In [25]:
import math
    #Numeric Python
import numpy as np
    #Pandas (dataframes)
import pandas as pd
    #datetime for fate manipulation
from datetime import date, datetime, timedelta  
    #Regex for advanced string matching
import re
    #for time related stuff
import time
    #json library
import json
    #Analyst tools
import sys
sys.path.append('~')
from analysts_tools.growth import *
    #Procurement tools
from analystcommunity.read_connection_data_warehouse import run_read_dwd_query
from analysts_tools.redash_methods import *
from procurement_lib import redash, dw, send_message, send_slack_notification, GoogleSheet, logging

import random
import time
import datetime

In [26]:
query = """
SELECT
    last_order_city AS city,
    source_id AS customer_id,
    recency_days AS recency_days,
    DATEDIFF(day, first_submit_date, GETDATE()) AS alive_days,
    CASE WHEN microsegment_name IS NULL THEN 'NO' ELSE microsegment_name END AS microsegment,
    CASE WHEN business_type IS NULL THEN 'NO' ELSE business_type END AS segment,
    CASE 
        WHEN is_kam_audited = 1 THEN 'KAM' 
        WHEN is_graduated = 1 THEN 'Grads'
        WHEN salesperson_mail <> 'No value' THEN 'Farming'
    ELSE 'Other' END AS canal--,
    --is_graduated AS grad,
    --is_kam_audited AS kam,
    --CASE WHEN salesperson_mail <> 'No value' THEN 1 ELSE 0 END AS is_farming
FROM dpr_customer.obt_customer

WHERE recency_days <= 180
"""

df = run_read_dwd_query(query)

In [27]:
def homes(city):
    customers =  df.loc[
                (df.city == city) &
                (df.segment.isin(['Hogar'])) &
                (~df.microsegment.isin(['ALL'])) &
                (~df.canal.isin(['ALL'])) &
                (df.alive_days <= 10000) &
                (df.recency_days <= 60) 
            ].customer_id.unique().tolist()
    return customers

In [28]:
def create_segment(city,customers,name,date):
  import requests
  import json

  url = f"https://{city}.frubana.com/api/v1/segments/create?api-key=00_growth-team-zzz-qqq_001"

  payload = json.dumps([
      {
      "name": name, 
      "customersIds": customers,
      "expiresAt": date
  }
      ]
                      )


  headers = {
    'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALB=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI; AWSALBCORS=jLY02FPysvF1vTrKLhXbk3ibzmyiAqFJ1ezYVu5PjodkU2QmPHDpRJhb9CvOlOyQ1Ix0mJR1ig+eH9zwJ9i5EOtgsgx79NSROogQb2Ua6yLX/I3bJJ6xtcVO3puI',
    'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)

  return response.text

In [29]:
df.city.unique()

array(['SPO', 'BOG', 'BAQ', 'BHZ', 'CMX', 'MDE', 'VCP', 'CWB'],
      dtype=object)

In [30]:
# for city in df.city.unique():
#     response = create_segment(city,homes(city),f"{city}_HOMES_2024",'2025-01-01')
#     print(response)

In [33]:
df_cayena=GoogleSheet("1tQZXQaLQqUF-Y5Ue8yQttcy-i5uiqPSSJpzpm1mS5gY")
df_cayena = df_cayena.get_as_dataframe('Dados Pedidos')

customers_cayena = df_cayena.loc[df_cayena['Cadastro Frubana?'] == 'Sim','ID'].unique().tolist()

In [32]:
customers = homes('SPO')

In [37]:
result_list = customers + customers_cayena

In [43]:
len(list(set(result_list)))

1686

In [44]:
city = 'SPO'

In [45]:
response = create_segment(city,list(set(result_list)),f"{city}_HOMES+CAYENA_2024",'2025-01-01')
print(response)

Customer segments were created: [577202-SPO_HOMES+CAYENA_2024]


In [ ]:
def upload_homes(city,customers,customerSegmentId):
  import requests
  import json

  url = f"https://{city}.frubana.com/api/v1/segments/massive/update?api-key=00_growth-team-zzz-qqq_001"

  payload = json.dumps({
    "customerSegmentId": customerSegmentId,
    "customersIds": customers
  })
  headers = {
    'Cookie': 'AWSALB=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1; AWSALBCORS=WGKrKvA1CFOY0m92xSENGAseuE4LrcfQh9y8jK/loGBJdizNm+FvjOqUqVghxdfLkH6Jsa9sfWft2NELchGAP/LNZMXMYhrOE70qDnUuOHATpdJj1UZBMEUBSIN1',
    'Content-Type': 'application/json'
  }

  response = requests.request("POST", url, headers=headers, data=payload)

  return response.text

In [ ]:
for city in df.city.unique():
    response = upload_homes(city,homes(city),1)
    print(response)